# Imports

In [1]:
from ipywidgets import Layout, Button, Label, Password, Text, VBox, HBox, Output, HTML, BoundedFloatText, BoundedIntText
from IPython.display import display
from dashchecks import *

# Dashboard

In [ ]:
# Different styles definitions:
login_button_style = {"description_border_color": "lightblue", "font_weight": "bold", "button_color": "lightblue"}
input_style = {"description_width": "initial"}
header_style = {"font_weight": "bold", "color": "black", "background-color": "lightblue", "padding": "10px", "text-align": "center"}
welcome_style = {"font_weight": "bold", "color": "#008080", "background-color": "lightblue", "padding": "10px", "text-align": "center", "font-size": "28px"}

# Log In Fields
username = Text(placeholder="Username", style=input_style)
password = Password(placeholder="Password", style=input_style)

# Create the buttons
login_button = Button(description="Log In", style=login_button_style, icon="check", button_style="")
sign_up_button = Button(description="Sign Up", style=login_button_style, icon="plus", button_style="")
back_button = Button(description="Back", style=login_button_style, icon="arrow-left", button_style="")
create_button = Button(description="Create", style=login_button_style, icon="check", button_style="")

# Create output widget
output = Output()

# Create the VirtualBox for the login
login_form = VBox([
    Label(value="Hello, welcome!", layout=Layout(margin="10px"), style=welcome_style),
    username,
    password,
    VBox([login_button], layout=Layout(justify_content="center")),
    VBox([sign_up_button], layout=Layout(justify_content="center")),
    output,
], layout=Layout(justify_content="center", align_items="center"))

# SignUp Fields
Employee_ID = Text(placeholder="Employee ID", style=input_style)
First_Name = Text(placeholder="First Name", style=input_style)
Last_Name = Text(placeholder="Last Name", style=input_style)
signup_username = Text(placeholder="Username", style=input_style)
signup_password = Password(placeholder="Password", style=input_style)

# Create the VirtualBox for the SugnUp
signup_form = VBox([
    Label(value="Create an Account", layout=Layout(margin="10px"), style=welcome_style),
    Employee_ID,
    First_Name,
    Last_Name,
    signup_username,
    signup_password,
    VBox([create_button], layout=Layout(justify_content="center")),
    VBox([back_button], layout=Layout(justify_content="center")),
], layout=Layout(justify_content="center", align_items="center"))

# Function to handle login button click
def login_button_clicked(button):
    form_container.children = [login_form]

# Function to handle signup button click
def signup_button_clicked(button):
    form_container.children = [signup_form]

# Function to handle back button click
def back_button_clicked(button):
    form_container.children = [login_form]

# Function to handle create button click
def create_button_clicked(button):
    # Placeholder--Come back to this later.
    with output:
        output.clear_output()
        print("Account created successfully!")

# Button clicks and their events.
login_button.on_click(login_button_clicked)
sign_up_button.on_click(signup_button_clicked)
back_button.on_click(back_button_clicked)
create_button.on_click(create_button_clicked)

# Create layout (Virtual Boxes)
def create_layout():
    login_form_box = VBox([
        Label(value="Hello, welcome!", layout=Layout(margin="10px"), style=welcome_style),
        username,
        password,
        VBox([login_button], layout=Layout(justify_content="center")),
        VBox([sign_up_button], layout=Layout(justify_content="center")),
        output,
    ], layout=Layout(justify_content="center", align_items="center"))
    
    signup_form_box = VBox([
        Label(value="Create an Account", layout=Layout(margin="10px"), style=welcome_style),
        Employee_ID,
        First_Name,
        Last_Name,
        signup_username,
        signup_password,
        VBox([create_button], layout=Layout(justify_content="center")),
        VBox([back_button], layout=Layout(justify_content="center")),
    ], layout=Layout(justify_content="center", align_items="center"))
    
    form_container = HBox([login_form_box], layout=Layout(justify_content="center", align_items="center"))
    
    header = HTML("<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><p style='font-weight:bold; color:black; font-size:24px;'>HotelAnalytics Systems</p><p style='font-weight:bold; color:black; font-size:18px;'>DB Beasts</p></div>") 
    footer = HTML("<div style='background-color:lightblue; padding:10px; text-align:center; overflow:hidden;'><a style='text-decoration:none; color:black;'>Db Beasts Members:</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/YanehRuiz' target='_blank' style='text-decoration:none; color:black;'> Yanishka</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/gaac-rum' target='_blank' style='text-decoration:none; color:black;'> Gabriel</a> <i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/Axel-Negron' target='_blank' style='text-decoration:none; color:black;'> Axel  </a><i class='fas fa-user' style='background-color: #00868B; color: white; border-radius: 50%; padding: 2px 6px; font-weight: bold;'></i><a href='https://github.com/samiralirivera' target='_blank' style='text-decoration:none; color:black;'> Samir</a></div>")    
    background = HTML("<img src='https://i.imgur.com/sJ8W01b.jpeg' style='width:100%; height:90vh; object-fit:cover;'>")   
    return header, form_container, background, footer

# Display header, layout, and backgrounde
header, form_container, background, footer = create_layout()
display(header)
display(form_container)
display(background)
display(footer)

## Crud

### Chain

In [5]:
from ipywidgets import Layout, Button, Label, Password, Text, VBox, HBox, Output, HTML, BoundedFloatText, BoundedIntText
from IPython.display import display
from dashchecks import *
#######################      Create Chain ##########################
def CreateChain():
    def post_chain(button):
        with output:
            chain = Post_Chain(cname.value, fallmkup.value, springmkup.value, summermkup.value, wintermkup.value)
            if chain.status_code == 201:
                output.clear_output()
                print("Chain created successfully")
                print("Chain Data:")
                print(chain.json())  # Print the chain data
            else:
                output.clear_output()
                print(f"Invalid Chain Data")

    operation = HTML(f"<p style='font-size:24px;'>Create Chain</p>")
    
    cname = Text(placeholder="---", description='Chain Name')
    fallmkup = BoundedFloatText(min=0, step=0.01, description='Fall Mkup')
    springmkup = BoundedFloatText(min=0, step=0.01, description='Spring Mkup')
    summermkup = BoundedFloatText(min=0, step=0.01, description='Summer Mkup')
    wintermkup = BoundedFloatText(min=0, step=0.01, description='Winter Mkup')
    create_button = Button(description='Create Chain')
    create_button.on_click(post_chain)
    
    create_box = VBox([operation, cname, fallmkup, springmkup, summermkup, wintermkup, create_button])
    display(create_box)
    
    output = Output()
    display(output)
    
display(CreateChain())

Output()

None

In [6]:
#######################      Delete Chain ##########################
def DeleteChain():
    def delete_chain(b):
        with output:
            chain = Delete_Chain(chid.value)
            if chain.status_code == 200:
                output.clear_output()
                print("Chain deleted successfully")
            elif chain.status_code == 404:
                output.clear_output()
                print("Chain not found")
            elif chain.status_code == 400:
                output.clear_output()
                print("Chain is referenced")
            else:
                output.clear_output()
                print("Delete Failed")
            
    operation = HTML(f"<p style='font-size:24px;'>Delete Chain</p>")
    
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    delete_button = Button(description="Delete Chain")
    delete_button.on_click(delete_chain)
    
    create_box = VBox([operation, chid, delete_button])
    display(create_box)
    
    output = Output()
    display(output)

display(DeleteChain())

Output()

None

In [7]:
#######################      Update Chain ##########################
def UpdateChain():
    def put_chain(b):
        with output:
            chain = Put_Chain(chid.value, cname.value, fallmkup.value, springmkup.value, summermkup.value, wintermkup.value)
            if chain.status_code == 202:
                output.clear_output()
                print("Chain updated successfully")
                print("Chain Data:")
                print(chain.json())
            elif chain.status_code == 404:
                output.clear_output()
                print("Chain not found")
            else:
                output.clear_output()
                print("Invalid Chain Data")
            
    operation = HTML(f"<p style='font-size:24px;'>Update Chain</p>")
    
    chid = BoundedIntText(min=-1,step=1, description="Chain Id")
    cname = Text(placeholder="---", description='Chain Name')
    fallmkup = BoundedFloatText(min=0, step=0.01, description='Fall Mkup')
    springmkup = BoundedFloatText(min=0, step=0.01, description='Spring Mkup')
    summermkup = BoundedFloatText(min=0, step=0.01, description='Summer Mkup')
    wintermkup = BoundedFloatText(min=0, step=0.01, description='Winter Mkup')
    update_button = Button(description="Update Chain")
    update_button.on_click(put_chain)
    
    create_box = VBox([operation, chid, cname, fallmkup, springmkup, summermkup, wintermkup, update_button])
    display(create_box)
    
    output = Output()
    display(output)
    
display(UpdateChain())

Output()

None

### Client

In [ ]:
#######################      Create Client ##########################

In [ ]:
#######################      Delete  ##########################

In [ ]:
#######################      Update Client ##########################

### Employee

In [ ]:
#######################      Create Employee ##########################

In [ ]:
#######################      Delete Employee ##########################

In [ ]:
#######################      Update Employee ##########################

### Hotel

In [ ]:
#######################      Create Hotel ##########################

In [ ]:
#######################      Delete Hotel ##########################

In [ ]:
#######################      Update Hotel ##########################

### Login

In [ ]:
#######################      Create Login ##########################

In [ ]:
#######################      Delete Login ##########################

In [ ]:
#######################      Update Login ##########################

### Reserve

In [ ]:
#######################      Create Reserve ##########################

In [ ]:
#######################      Delete Reserve ##########################

In [ ]:
#######################      Update Reserve ##########################

### Room

In [ ]:
#######################      Create Room ##########################

In [ ]:
#######################      Delete Room ##########################

In [ ]:
#######################      Update Room ##########################

### Roomdescription

In [ ]:
#######################      Create Roomdescription ##########################

In [ ]:
#######################      Delete Roomdescription ##########################

In [ ]:
#######################      Update Roomdescription ##########################

### Roomunavailable

In [ ]:
#######################      Create Roomunavailable ##########################

In [ ]:
#######################      Delete Roomunavailable ##########################

In [ ]:
#######################      Update Roomunavailable ##########################

## Globales

In [ ]:
#######################     /most/revenue ##########################

In [ ]:
#######################      /paymentmethod ##########################

In [ ]:
#######################     /least/rooms ##########################

In [ ]:
#######################      /most/capacity ##########################

In [ ]:
#######################      /most/reservation ##########################

In [ ]:
#######################      /most/profitmonth ##########################

## Locales

In [ ]:
#######################      handicaproom ##########################

In [ ]:
#######################      leastreserve ##########################

In [ ]:
#######################      mostcreditcard ##########################

In [ ]:
#######################      highestpaid ##########################

In [ ]:
#######################      mostdiscount ##########################

In [ ]:
#######################      roomtype ##########################

In [ ]:
#######################      leastguests ##########################